In [29]:
# Function to add pauses into transcript

def stitch_up_transcript(segments, pause_threshold=3):
    if not segments:
        return ""

    stitched = segments[0]['text'].strip()
    for i in range(len(segments) - 1):
        gap = segments[i+1]['start'] - segments[i]['end']
        if gap > pause_threshold:
            stitched += " ... "
        else:
            stitched += " "
        stitched += segments[i+1]['text'].strip()

    return stitched

In [30]:
# Format transcript from whisper and put into json format

def format_transcript(patient_id, day_num, result, pause_threshold=0.5):
    segments = result.get('segments', [])

    transcript_text = stitch_up_transcript(segments, pause_threshold)

    duration = segments[-1]['end'] if segments else 0

    return {
        'patient_id': patient_id,
        'day_num': day_num,
        'duration_sec': duration,
        'transcript_text': transcript_text,
        'segments': [
            {
                'start': segment['start'],
                'end': segment['end'],
                'text': segment['text']
            }
            for segment in segments
        ]
    }

In [31]:
# Whisper model to transcribe audio

import whisper

AUDIO_PATH = 'test_audio_full.wav'
model = whisper.load_model('tiny')

result = model.transcribe(AUDIO_PATH, word_timestamps=True)

/Users/arjunpalakkal/Documents/[01] Personal/[01] Coding Projects/[02] Github Stuff/[02] Alzheimers/.venv/lib/python3.10/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


KeyboardInterrupt: 

In [32]:
# Dump formatted json into a file

json_data = format_transcript("P001", 8, result)
import json
with open('transcription.json', 'w') as f:
    json.dump(json_data, f, indent=4)